In [ ]:
# ==========================================================
# BCS Survey Logic Checker (with row-level details)
# ==========================================================
import re
import numpy as np
import pandas as pd
import streamlit as st
import json
import csv
from io import BytesIO
import io

# -------------------------------------------------------------------
# App setup + theme
# -------------------------------------------------------------------
st.set_page_config(page_title="BCS Survey Logic Checker", layout="wide")

def set_background_solid(main="#F4EB89", sidebar="#EEEFF3"):
    st.markdown(f"""
    <style>
      [data-testid="stAppViewContainer"],
      [data-testid="stAppViewContainer"] .main,
      [data-testid="stAppViewContainer"] .block-container {{
        background-color: {main} !important;
      }}
      [data-testid="stSidebar"],
      [data-testid="stSidebar"] > div,
      [data-testid="stSidebar"] .block-container {{
        background-color: {sidebar} !important;
      }}
      header[data-testid="stHeader"] {{ background: transparent; }}
      [data-testid="stDataFrame"],
      [data-testid="stTable"] {{ background-color: transparent !important; }}
    </style>
    """, unsafe_allow_html=True)

set_background_solid()
st.title("📊 BCS Survey Logic Checker")
st.caption("This tool is specifically designed for BCS Thailand/Taiwan. Identified mismatches will be highlighted in the deliverables.")

# -------------------------------------------------------------------
# File helpers
# -------------------------------------------------------------------
COMMON_ENCODINGS = ["utf-8", "utf-8-sig", "cp1252", "latin-1"]
ZIP_SIGNATURES = (b"PK\x03\x04", b"PK\x05\x06", b"PK\x07\x08")

def _sniff_sep(sample_text: str) -> str:
    try:
        dialect = csv.Sniffer().sniff(sample_text[:4096], delimiters=",;\t|")
        return dialect.delimiter
    except Exception:
        return ","

def _norm_delim(sel: str) -> str:
    return {"\\t": "\t"}.get(sel, sel)

def read_any_table(uploaded_file, enc_override="auto", delim_override="auto", skip_bad=True) -> pd.DataFrame:
    name = (uploaded_file.name or "").lower()
    raw = uploaded_file.read()

    if raw.startswith(ZIP_SIGNATURES) or name.endswith((".xlsx", ".xls")):
        uploaded_file.seek(0)
        return pd.read_excel(uploaded_file)

    encodings = COMMON_ENCODINGS if enc_override == "auto" else [enc_override]
    for enc_try in encodings:
        try:
            text = raw.decode(enc_try, errors="strict")
            sep = _sniff_sep(text) if delim_override == "auto" else _norm_delim(delim_override)
            kwargs = dict(encoding=enc_try, sep=sep, engine="python")
            if skip_bad:
                kwargs["on_bad_lines"] = "skip"
            return pd.read_csv(BytesIO(raw), **kwargs)
        except Exception:
            continue

    sep = "," if delim_override == "auto" else _norm_delim(delim_override)
    kwargs = dict(encoding="latin-1", sep=sep, engine="python")
    if skip_bad:
        kwargs["on_bad_lines"] = "skip"
    return pd.read_csv(BytesIO(raw), **kwargs)

# -------------------------------------------------------------------
# Sidebar upload
# -------------------------------------------------------------------
with st.sidebar:
    st.header("Input")
    data_file = st.file_uploader("Current wave data", type=["csv", "xlsx", "xls"])
    rules_file = st.file_uploader("Optional: custom rules JSON", type=["json"])
    rules = json.load(rules_file) if rules_file else None

    st.markdown("---")
    st.subheader("Parser overrides")
    enc = st.selectbox("Encoding", ["auto", "utf-8", "utf-8-sig", "cp1252", "latin-1"], index=0)
    delim = st.selectbox("Delimiter", ["auto", ",", ";", "\\t", "|"], index=0)
    skip_bad = st.checkbox("Skip bad lines", value=True)

if not data_file:
    st.info("Upload a CSV/XLSX to begin.")
    st.stop()

try:
    data_file.seek(0)
    df = read_any_table(data_file, enc_override=enc, delim_override=delim, skip_bad=skip_bad)
except Exception as e:
    st.error(f"Failed to read file: {e}")
    st.stop()

# -------------------------------------------------------------------
# Clean null tokens
# -------------------------------------------------------------------
df.replace(
    {"#NULL!": np.nan, "NULL": np.nan, "null": np.nan, "NaN": np.nan, "nan": np.nan,
     "": np.nan, "na": np.nan, "N/A": np.nan, "n/a": np.nan},
    inplace=True,
)

# -------------------------------------------------------------------
# Rules list (global only)
# -------------------------------------------------------------------
SURVEY_RULES = {
    1: "Main brand must exist",
    2: "Quota make autocoded",
    3: "Company position requires OE if 98",
    4: "Fleet size numeric (0–99999), terminate if 0",
    5: "Last purchase required if S3>0",
    6: "Usage single brand → main_brand must match",
    10: "Quota make satisfaction set required",
    11: "Truck defects → require OE",
    12: "Volvo quota → require satisfaction & dissatisfaction comments",
    13: "Barriers → require follow-ups",
    14: "Transport type=98 → require OE",
    15: "Quota make in Volvo group → require operation range",
    16: "System fields (region, country, survey_year) required",
}

# -------------------------------------------------------------------
# Check engine (row-level)
# -------------------------------------------------------------------
digest = []
detailed = []

def add_issue(rule_id, msg, idx=None):
    digest.append((rule_id, msg))
    if idx is not None:
        detailed.append((idx, rule_id, msg))

# Rule 1 – main_brand
if "main_brand" not in df.columns:
    add_issue(1, "Missing main_brand column")
else:
    if df["main_brand"].isna().any():
        bad = df[df["main_brand"].isna()].index
        for i in bad: add_issue(1, "main_brand missing", i)

# Rule 2 – quota_make
if "quota_make" not in df.columns:
    add_issue(2, "Missing quota_make column")
elif "main_brand" in df.columns:
    bad = df["quota_make"].astype(str) != df["main_brand"].astype(str)
    for i in df[bad].index: add_issue(2, "quota_make ≠ main_brand", i)

# Rule 3 – company_position
if "company_position" not in df.columns:
    add_issue(3, "Missing company_position")
else:
    if (df["company_position"] == 98).any():
        if "company_position_other_specify" not in df.columns:
            for i in df[df["company_position"]==98].index:
                add_issue(3, "Missing OE for company_position=98", i)

# Rule 4 – fleet size
if "n_heavy_duty_trucks" not in df.columns:
    add_issue(4, "Missing n_heavy_duty_trucks")
else:
    vals = pd.to_numeric(df["n_heavy_duty_trucks"], errors="coerce")
    for i in df[vals.isna()].index: add_issue(4, "Invalid numeric S3", i)
    for i in df[(vals < 0) | (vals > 99999)].index: add_issue(4, "S3 out of range", i)
    for i in df[vals==0].index: add_issue(4, "S3=0 (terminate)", i)

# Rule 5 – last_purchase_hdt
if "last_purchase_hdt" not in df.columns:
    add_issue(5, "Missing last_purchase_hdt")

# Rule 6 – usage vs main_brand
usage_cols = [c for c in df.columns if c.startswith("usage_")]
if usage_cols and "main_brand" in df.columns and "A2b" in df.columns:
    one_brand = df[usage_cols].sum(axis=1) == 1
    bad = one_brand & (df["A2b"].astype(str) != df["main_brand"].astype(str))
    for i in df[bad].index: add_issue(6, "A2b ≠ single usage brand", i)

# Rule 10 – quota satisfaction set
quota_checks = ["overall_satisfaction","likelihood_choose_brand","likelihood_choose_workshop","preference_strength","overall_rating_truck"]
for c in quota_checks:
    if c not in df.columns:
        add_issue(10, f"Missing {c}")

# Rule 11 – truck defects
if "truck_defects" in df.columns and (df["truck_defects"]==1).any():
    if "truck_defects_other_specify" not in df.columns:
        for i in df[df["truck_defects"]==1].index:
            add_issue(11, "Missing OE for truck_defects=1", i)

# Rule 12 – Volvo quota
if "quota_make" in df.columns and (df["quota_make"].astype(str)=="38").any():
    for c in ["satisfaction_comments","dissatisfaction_comments"]:
        if c not in df.columns:
            for i in df[df["quota_make"].astype(str)=="38"].index:
                add_issue(12, f"Missing {c} for Volvo", i)

# Rule 13 – Barriers
if "reasons_not_consider_volvo" in df.columns:
    for follow, col in [("a","a_barriers_follow_up"),("b","b_barriers_follow_up"),("c","c_barriers_follow_up")]:
        if col not in df.columns:
            for i in df.index: add_issue(13, f"Missing {col}", i)

# Rule 14 – transport_type OE
if "transport_type" in df.columns and (df["transport_type"]==98).any():
    if "transport_type_other_specify" not in df.columns:
        for i in df[df["transport_type"]==98].index:
            add_issue(14, "Missing OE for transport_type=98", i)

# Rule 15 – Volvo group operation range
if "quota_make" in df.columns and df["quota_make"].astype(str).isin(["38","31","23","9"]).any():
    if "operation_range_volvo_hdt" not in df.columns:
        for i in df[df["quota_make"].astype(str).isin(["38","31","23","9"])].index:
            add_issue(15, "Missing operation_range_volvo_hdt", i)

# Rule 16 – system fields
for sysc in ["region","country","survey_year"]:
    if sysc not in df.columns:
        add_issue(16, f"Missing {sysc}")

# -------------------------------------------------------------------
# Prepare outputs
# -------------------------------------------------------------------
digest_df = pd.DataFrame(digest, columns=["RuleID","Issue"]).drop_duplicates()
detailed_df = pd.DataFrame(detailed, columns=["RowID","RuleID","Issue"])

st.subheader("Survey Logic Issues")
if digest_df.empty:
    st.success("✅ No issues found – dataset follows survey logic.")
else:
    st.dataframe(digest_df, use_container_width=True)

    # Export to Excel
    out = io.BytesIO()
    with pd.ExcelWriter(out, engine="xlsxwriter") as writer:
        digest_df.to_excel(writer, index=False, sheet_name="Digest")
        detailed_df.to_excel(writer, index=False, sheet_name="Detailed")
    st.download_button(
        "📥 Download Issues (Excel)",
        data=out.getvalue(),
        file_name="survey_logic_issues.xlsx",
        mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
    )
